In [23]:
import os
import pandas as pd
import cv2

In [24]:
ROOT_DIR = os.path.join(os.getcwd(), os.pardir)
DATA_DIR = os.path.join(ROOT_DIR, 'data')
TRAIN_VIDEOS_DIR = os.path.join(DATA_DIR, 'train_videos')
MARKUP_DIR = os.path.join(DATA_DIR, 'markup_df_train.xlsx')
VIDEOS_DIR = os.path.join(DATA_DIR, 'train_videos')

print(MARKUP_DIR)
print(VIDEOS_DIR)

/home/gotheartem/Projects/traffic-violations-detection/notebooks/../data/markup_df_train.xlsx
/home/gotheartem/Projects/traffic-violations-detection/notebooks/../data/train_videos


In [25]:
video_paths = []

# get video paths
for filename in os.listdir(TRAIN_VIDEOS_DIR):
    # ignore txt
    if filename.endswith('.txt'):
        continue

    video_path = os.path.join(TRAIN_VIDEOS_DIR, filename)
    video_paths.append(video_path)

In [ ]:
violation_df = pd.DataFrame(columns=['id', 'time', 'violation'])
for filename in video_paths:
    # захватываем видео
    video = cv2.VideoCapture(filename)

    # считаем продолжительность в секундах
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = round(video.get(cv2.CAP_PROP_FPS))
    duration = round(total_frames / fps)

    # получаем имя файла и обрезаем расширение
    filename = filename.split(os.path.sep)[-1]
    filename = filename[:filename.find('.')]

    # создаем df для этого видео
    video_df = pd.DataFrame({'id': filename.split(os.path.sep)[-1].lower(), 
                             'time': range(1, duration + 1),
                             'violation': 'nothing'})
    
    violation_df = pd.concat((violation_df, video_df), axis=0)

In [27]:
violation_df

,id,time,violation
0,akn00088,1,nothing
1,akn00088,2,nothing
2,akn00088,3,nothing
3,akn00088,4,nothing
4,akn00088,5,nothing
...,...,...,...
295,akn00006_fqgg6dtl,296,nothing
296,akn00006_fqgg6dtl,297,nothing
297,akn00006_fqgg6dtl,298,nothing
298,akn00006_fqgg6dtl,299,nothing


In [28]:
# Чтение файла Excel в DataFrame
markup_df = pd.read_excel(MARKUP_DIR)
markup_df.head()

,номер видео,наименование нарушения,"сумма штрафа, руб.",время нарушения (в секундах)
0,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,18
1,akn00006_fqGg6dtL,Статья 12.16 часть 2 Поворот налево или развор...,1000,75
2,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,249
3,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,250
4,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,263


In [29]:
for idx, row in markup_df.iterrows():
    video_id = row.iloc[0].lower() # номер видео
    violation = row.iloc[1] # наименование нарушения
    violation_time = row.iloc[3] # время нарушения

    # получаем продолжительность видео
    video_duration = violation_df[violation_df['id'] == video_id]['time'].max()
    window_start = max(1, violation_time - 10) # начало временного окна в котором ищем нарушение
    window_end = min(video_duration, violation_time + 10) # конец временного окна в котором ищем нарушение

    # выбираем записи с данным video_id, временное окно и устанавливаем наименование нарушения
    mask = violation_df['id'] == video_id
    violation_df.loc[mask & (violation_df.index >= window_start) & (violation_df.index < window_end), 'violation'] = violation

In [30]:
violation_df = violation_df.reset_index(drop=True)

In [31]:
violation_df

,id,time,violation
0,akn00088,1,nothing
1,akn00088,2,nothing
2,akn00088,3,nothing
3,akn00088,4,nothing
4,akn00088,5,nothing
...,...,...,...
2995,akn00006_fqgg6dtl,296,nothing
2996,akn00006_fqgg6dtl,297,nothing
2997,akn00006_fqgg6dtl,298,nothing
2998,akn00006_fqgg6dtl,299,nothing


In [32]:
violation_df[violation_df['violation'] != 'nothing']

,id,time,violation
226,akn00088,227,Статья 12.15 часть 4 Выезд в нарушение правил ...
227,akn00088,228,Статья 12.15 часть 4 Выезд в нарушение правил ...
228,akn00088,229,Статья 12.15 часть 4 Выезд в нарушение правил ...
229,akn00088,230,Статья 12.15 часть 4 Выезд в нарушение правил ...
230,akn00088,231,Статья 12.15 часть 4 Выезд в нарушение правил ...
...,...,...,...
2983,akn00006_fqgg6dtl,284,Статья 12.17 часть 1.1 и 1.2. движение трансп...
2984,akn00006_fqgg6dtl,285,Статья 12.17 часть 1.1 и 1.2. движение трансп...
2985,akn00006_fqgg6dtl,286,Статья 12.17 часть 1.1 и 1.2. движение трансп...
2986,akn00006_fqgg6dtl,287,Статья 12.17 часть 1.1 и 1.2. движение трансп...


In [33]:
violation_df.to_csv('../data/train_targets.csv', index=False)